# Categorize Github support issues
Based on Microsoft's [ML.NET tutorial](https://docs.microsoft.com/en-us/dotnet/machine-learning/tutorials/github-issue-classification). It has been altered to accept set of JSON samples with the order of the issue randomized. 

In [1]:
#r "nuget:Microsoft.ML"

In [1]:
using System.IO;
using System.Linq;
using System.Text.Json;
using Microsoft.ML;
using Microsoft.ML.Data;

In [1]:
public class GitHubIssue
{
    [LoadColumn(0)]
    public string ID { get; set; }
    [LoadColumn(1)]
    public string Area { get; set; }
    [LoadColumn(2)]
    public string Title { get; set; }
    [LoadColumn(3)]
    public string Description { get; set; }
}

public class IssuePrediction
{
    [ColumnName("PredictedLabel")]
    public string Area;
}

In [1]:
var trainDataPath = Path.Combine(Environment.CurrentDirectory, "data", "issues_train.tsv");
var testDataPath = Path.Combine(Environment.CurrentDirectory, "data", "issues_test.tsv");
var modelPath = Path.Combine(Environment.CurrentDirectory, "models", "gh_issues.zip");

In [1]:
var ctx = new MLContext(seed: 0);
var trainingDataView = ctx.Data.LoadFromTextFile<GitHubIssue>(trainDataPath, hasHeader: true);

In [1]:
PredictionEngine<GitHubIssue, IssuePrediction> predEngine;
ITransformer trainedModel;

In [1]:
IEnumerable<GitHubIssue> SampleData(string file) 
{
    var data = Path.Combine(Environment.CurrentDirectory, "samples", file);
    var reader = File.ReadAllText(data);
    var opt = new JsonSerializerOptions 
    { 
        IgnoreNullValues = true, 
        PropertyNameCaseInsensitive = true,
    };
    var json = JsonSerializer.Deserialize<IEnumerable<GitHubIssue>>(reader, opt);
    var rng = new Random();
    var order = json.OrderBy(x => rng.Next(json.ToArray().Length));
    return order;
}

In [1]:
IEstimator<ITransformer> ProcessData(MLContext ctx)
{
    var pipeline = ctx.Transforms.Conversion.MapValueToKey(inputColumnName: "Area", outputColumnName: "Label")
    .Append(ctx.Transforms.Text.FeaturizeText(
    inputColumnName: "Title", outputColumnName: "TitleFeaturized"))
    .Append(ctx.Transforms.Text.FeaturizeText(inputColumnName: "Description", outputColumnName: "DescriptionFeaturized"))
    .Append(ctx.Transforms.Concatenate("Features", "TitleFeaturized", "DescriptionFeaturized"))
    .AppendCacheCheckpoint(ctx);

    return pipeline;
}

In [1]:
void BuildAndTrainModel(MLContext ctx, IDataView trainingDataView, IEstimator<ITransformer> pipeline, IEnumerable<GitHubIssue> sample)
{
    var training = pipeline.Append(ctx.MulticlassClassification.Trainers.SdcaMaximumEntropy("Label", "Features"))
    .Append(ctx.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

    Console.WriteLine("Training model ...");
    trainedModel = training.Fit(trainingDataView);
    Console.WriteLine($"Training ended: {DateTimeOffset.Now.ToString()}");

    predEngine = ctx.Model.CreatePredictionEngine<GitHubIssue, IssuePrediction>(trainedModel);

    foreach (var prediction in sample)
        predEngine.Predict(prediction);

}

In [1]:
void SaveModelAsFile(MLContext ctx, DataViewSchema trainingDataViewSchema, ITransformer model)
{
    ctx.Model.Save(model, trainingDataViewSchema, modelPath);
}

In [1]:
void Evaluate(MLContext ctx, DataViewSchema trainingDataViewSchema)
{   
    var testDataView = ctx.Data.LoadFromTextFile<GitHubIssue>(testDataPath, hasHeader: true);
    var testMetrics = ctx.MulticlassClassification.Evaluate(trainedModel.Transform(testDataView));

    Console.WriteLine($"* Metrics for Multi-class Classification model - Test Data");
    Console.WriteLine($"* MicroAccuracy: {testMetrics.MicroAccuracy:0.###}");
    Console.WriteLine($"* MacroAccuracy: {testMetrics.MacroAccuracy:0.###}");
    Console.WriteLine($"* LogLoss: {testMetrics.LogLoss:#.###}");
    Console.WriteLine($"* LogLossReduction: {testMetrics.LogLossReduction:#.###}");

    SaveModelAsFile(ctx, trainingDataViewSchema, trainedModel);
}

In [1]:
void PredictIssue(MLContext ctx, IEnumerable<GitHubIssue> sample)
{
    var loadedModel = ctx.Model.Load(modelPath, out var modelInputSchema);
    predEngine = ctx.Model.CreatePredictionEngine<GitHubIssue, IssuePrediction>(loadedModel);

    foreach (var prediction in sample)
    {
        var predict = predEngine.Predict(prediction);
        Console.WriteLine($"Predicting issue: {prediction.Title}{Environment.NewLine}Single Prediction - Result: {predict.Area}");
    }
}

In [1]:
var pipeline = ProcessData(ctx);
var trainingSamples = SampleData("issues_train.json");
BuildAndTrainModel(ctx, trainingDataView, pipeline, trainingSamples);

Error: System.IO.FileNotFoundException: Could not find file 'c:\Users\zc456\source\repos\mlworkshop\notebooks\samples\issuestrain.json'.
File name: 'c:\Users\zc456\source\repos\mlworkshop\notebooks\samples\issuestrain.json'
   at System.IO.FileStream.ValidateFileHandle(SafeFileHandle fileHandle)
   at System.IO.FileStream.CreateFileOpenHandle(FileMode mode, FileShare share, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options)
   at System.IO.StreamReader.ValidateArgsAndOpenPath(String path, Encoding encoding, Int32 bufferSize)
   at System.IO.StreamReader..ctor(String path, Encoding encoding, Boolean detectEncodingFromByteOrderMarks)
   at System.IO.File.InternalReadAllText(String path, Encoding encoding)
   at System.IO.File.ReadAllText(String path)
   at Submission#23.SampleData(String file)
   at Submission#32.<<Initialize>>d__0.MoveNext()
--- End of stack trace from pr

In [1]:
Evaluate(ctx, trainingDataView.Schema);

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#27.Evaluate(MLContext ctx, DataViewSchema trainingDataViewSchema)
   at Submission#30.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [1]:
var predictSamples = SampleData("issues_predict.json");
PredictIssue(ctx, predictSamples);

Error: System.IO.FileNotFoundException: Could not find file 'c:\Users\zc456\source\repos\mlworkshop\notebooks\samples\issuespredict.json'.
File name: 'c:\Users\zc456\source\repos\mlworkshop\notebooks\samples\issuespredict.json'
   at System.IO.FileStream.ValidateFileHandle(SafeFileHandle fileHandle)
   at System.IO.FileStream.CreateFileOpenHandle(FileMode mode, FileShare share, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options)
   at System.IO.StreamReader.ValidateArgsAndOpenPath(String path, Encoding encoding, Int32 bufferSize)
   at System.IO.StreamReader..ctor(String path, Encoding encoding, Boolean detectEncodingFromByteOrderMarks)
   at System.IO.File.InternalReadAllText(String path, Encoding encoding)
   at System.IO.File.ReadAllText(String path)
   at Submission#23.SampleData(String file)
   at Submission#31.<<Initialize>>d__0.MoveNext()
--- End of stack trace fro